In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import GridSearchCV, cross_val_score
#from alg7 import model

In [5]:
# explicitly require this experimental feature
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
# now you can import normally from ensemble
from sklearn.ensemble import HistGradientBoostingClassifier

model = HistGradientBoostingClassifier()

In [6]:
DATA='ugrin2020-vehiculo-usado-multiclase/'
TRAIN=DATA+'train.csv'
TEST=DATA+'test.csv'

PREPROCESSED_DATA='preprocessed_data/'
RESULTS='results/'

In [7]:
# Carga de datos ya procesados
X=np.load(PREPROCESSED_DATA+'binScale.npz')
train = X['arr_0']
label = X['arr_1']
test = X['arr_2']
X.close()

In [8]:
train.shape

(9125, 52)

In [11]:
param_grid={'learning_rate':[0.1,0.2,0.3,0.4,0.5,0.6],'max_iter':[100,200,300,400,500]}
searcher = GridSearchCV(model, param_grid, n_jobs=4,verbose=15,cv=3)
search = searcher.fit(train,label)

Fitting 3 folds for each of 30 candidates, totalling 90 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done   1 tasks      | elapsed:    7.5s
[Parallel(n_jobs=4)]: Done   2 tasks      | elapsed:    7.9s
[Parallel(n_jobs=4)]: Done   3 tasks      | elapsed:    7.9s
[Parallel(n_jobs=4)]: Done   4 tasks      | elapsed:   14.3s
[Parallel(n_jobs=4)]: Done   5 tasks      | elapsed:   20.5s
[Parallel(n_jobs=4)]: Done   6 tasks      | elapsed:   21.6s
[Parallel(n_jobs=4)]: Done   7 tasks      | elapsed:   28.2s
[Parallel(n_jobs=4)]: Done   8 tasks      | elapsed:   33.4s
[Parallel(n_jobs=4)]: Done   9 tasks      | elapsed:   43.6s
[Parallel(n_jobs=4)]: Done  10 tasks      | elapsed:   47.3s
[Parallel(n_jobs=4)]: Done  11 tasks      | elapsed:   55.4s
[Parallel(n_jobs=4)]: Done  12 tasks      | elapsed:   56.9s
[Parallel(n_jobs=4)]: Done  13 tasks      | elapsed:  1.1min
[Parallel(n_jobs=4)]: Done  14 tasks      | elapsed:  1.2min
[Parallel(n_jobs=4)]: Done  15 tasks      | elapsed:  1.3min
[Parallel(

In [12]:
df=pd.DataFrame(search.cv_results_)[['param_learning_rate','param_max_iter','mean_test_score','rank_test_score']].sort_values(by='rank_test_score')
with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
    display(df)

,param_learning_rate,param_max_iter,mean_test_score,rank_test_score
1,0.1,200,0.924274,1
10,0.3,100,0.923945,2
2,0.1,300,0.923726,3
0,0.1,100,0.923069,4
5,0.2,100,0.923068,5
14,0.3,500,0.922520,6
13,0.3,400,0.922520,7
17,0.4,300,0.922192,8
3,0.1,400,0.922192,8
11,0.3,200,0.921972,10


In [ ]:
print('Mejores parámetros:', search.best_params_)
print('Mejor puntuación:', search.best_score_)
model.set_params(**(search.best_params_))

In [ ]:
scores=cross_val_score(model, train, label)
print(scores)
print(np.mean(scores))

In [ ]:
model.fit(train,label)